In [28]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
%ls

 EmotionDetectionDeep.h5     fer2013.bib   README
 EmotionDetectionDeep.json   fer2013.csv  'Untitled folder'/


In [30]:
%cd 'drive'
%ls 

[Errno 2] No such file or directory: 'drive'
/content/drive/My Drive/fer2013
 EmotionDetectionDeep.h5     fer2013.bib   README
 EmotionDetectionDeep.json   fer2013.csv  'Untitled folder'/


In [31]:
%cd 'My Drive'
%ls

[Errno 2] No such file or directory: 'My Drive'
/content/drive/My Drive/fer2013
 EmotionDetectionDeep.h5     fer2013.bib   README
 EmotionDetectionDeep.json   fer2013.csv  'Untitled folder'/


In [32]:
%cd fer2013

[Errno 2] No such file or directory: 'fer2013'
/content/drive/My Drive/fer2013


In [33]:
'''importing all the necessary libraries'''
import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils


In [34]:
'''read the file in a pandas dataframe'''
df=pd.read_csv('fer2013.csv')

In [74]:
'''make lists for training and testing sets'''
X_train,Y_train,X_test,Y_test=[],[],[],[]

'''our initial pixel array has a type of object convert that to float32 and convert each row appending to training and testing lists to a numpy array'''
for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           Y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           Y_test.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


num_labels = 7
batch_size = 64
epochs = 28
width, height = 48, 48

'''convert the whole training and testing list into a numpy array'''
X_train = np.array(X_train,'float32')
Y_train = np.array(Y_train,'float32')
X_test = np.array(X_test,'float32')
Y_test = np.array(Y_test,'float32')

'''categorical_crossentropy requires the data in categorical form'''
Y_train=np_utils.to_categorical(Y_train, num_classes=num_labels)
Y_test=np_utils.to_categorical(Y_test, num_classes=num_labels)

'''Normalize the X inputs'''
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

'''Convert the X inputs into required form for a Neural Network'''
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)


In [75]:
print("X_train:{}".format(X_train[0:2]))
print("X_test:{}".format(X_test[0:2]))
print("Y_train:{}".format(Y_train[0:2]))
print("Y_test:{}".format(X_test[0:2]))

X_train:[[[[-0.6098866 ]
   [-0.4592209 ]
   [-0.40325198]
   ...
   [-0.7694696 ]
   [-0.90518403]
   [-0.95160526]]

  [[-0.66049284]
   [-0.68162924]
   [-0.694159  ]
   ...
   [-0.7112763 ]
   [-0.7862608 ]
   [-0.90819967]]

  [[-0.83170736]
   [-0.8952668 ]
   [-0.73603356]
   ...
   [-0.7924775 ]
   [-0.7261497 ]
   [-0.8613453 ]]

  ...

  [[-0.33269772]
   [-0.6514918 ]
   [-0.93670547]
   ...
   [-0.5265123 ]
   [-0.7377333 ]
   [-0.9078971 ]]

  [[-0.5099617 ]
   [-0.43692893]
   [-0.46537215]
   ...
   [-0.10019822]
   [-0.5580577 ]
   [-0.86989194]]

  [[-0.5095401 ]
   [-0.56317693]
   [-0.3986179 ]
   ...
   [-0.08629682]
   [-0.0573744 ]
   [-0.41127437]]]


 [[[ 0.37236458]
   [ 0.40461555]
   [ 0.41102034]
   ...
   [ 0.19833209]
   [ 0.2950228 ]
   [ 0.01161678]]

  [[ 0.3893324 ]
   [ 0.4109821 ]
   [ 0.45377484]
   ...
   [ 0.12469399]
   [ 0.3214692 ]
   [ 0.2319735 ]]

  [[ 0.40903383]
   [ 0.4540902 ]
   [ 0.56116307]
   ...
   [-0.02652   ]
   [ 0.11207935]
   

In [76]:
from tensorflow import keras
keras.backend.image_data_format()
keras.backend.set_image_data_format('channels_last')

In [77]:
'''Define the model architecture '''
layers = [
          Conv2D(filters=16, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu',input_shape=(width,height,1)),
          BatchNormalization(),
          MaxPooling2D(pool_size=(3,3),strides=(2,2)),
          Conv2D(filters=16, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu',input_shape=(width,height,1)),
          BatchNormalization(),
          Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'),
          BatchNormalization(),
          Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'),
          BatchNormalization(),
          MaxPooling2D(pool_size=(3,3),strides=(2,2)),
          Conv2D(filters=64, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'),
          BatchNormalization(),
          Conv2D(filters=64, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'),
          BatchNormalization(),
          MaxPooling2D(pool_size=(3,3),strides=(2,2)),
          Conv2D(filters=128, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'),
          BatchNormalization(),
          Conv2D(filters=128, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'),
          BatchNormalization(),
          MaxPooling2D(pool_size=(2,2),strides=(1,1)),
          Conv2D(filters=256, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'),
          BatchNormalization(),
          Conv2D(filters=256, kernel_size=(3,3), strides=(1, 1), padding='same', activation='relu'),
          BatchNormalization(),
          MaxPooling2D(pool_size=(2,2),strides=(2,2)),
          Flatten(),
          Dense(128,activation='relu'),
          Dense(64,activation='relu'),
          Dense(7,activation='softmax') ]

model = Sequential(layers)

model.summary()

'''Compile the model'''
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

'''Fit the model'''
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, Y_test),
          shuffle=True)


'''Save the model weights into a h5 file'''
save_json = model.to_json()
with open('EmotionDetectionDeep.json','w') as json_file:
    json_file.write(save_json )
model.save('EmotionDetectionDeep.h5')

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_154 (Conv2D)          (None, 48, 48, 16)        160       
_________________________________________________________________
batch_normalization_154 (Bat (None, 48, 48, 16)        64        
_________________________________________________________________
max_pooling2d_122 (MaxPoolin (None, 23, 23, 16)        0         
_________________________________________________________________
conv2d_155 (Conv2D)          (None, 23, 23, 16)        2320      
_________________________________________________________________
batch_normalization_155 (Bat (None, 23, 23, 16)        64        
_________________________________________________________________
conv2d_156 (Conv2D)          (None, 23, 23, 32)        4640      
_________________________________________________________________
batch_normalization_156 (Bat (None, 23, 23, 32)      

In [38]:
cd ..

/content/drive/My Drive


In [42]:
import cv2
img = cv2.imread('images-115.jpeg', cv2.IMREAD_GRAYSCALE)
img_r = cv2.resize(img,(48,48))

error: ignored

In [ ]:
import cv2
cv2.imread()
cv2.resize()
model.evaluate('data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxIQEhUSDxIVFRUVFRUVFRUVFRUVFRcVFRUWFhUWFRUYHSggGBolHRYVITEhJSkrLi4uFx8zODMtNygtLisBCgoKDg0OGBAQGC0dHR8rLS0tLSstLSsrLS0tKy0rLS0tLS0tLS0tLS0tLS0tLS0tLSstLS0rLS0rLS0rLS0tLf/AABEIANMA7wMBIgACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAADBAIFAQYHAAj/xAA8EAACAQICBwUGAwcFAQAAAAABAgADEQQhBQYSMUFRcSJhgZGxE3KhwdHwBzJSM0JigqKy8RQjksLhk//EABkBAQADAQEAAAAAAAAAAAAAAAABAgMEBf/EACQRAQEAAgICAgICAwAAAAAAAAABAhEDIRIxQXFRYTKRBBMi/9oADAMBAAIRAxEAPwDRaaRqkkhTSOUkkCVOnGqVOepJG6VOQM0qcbp05ilTjlOnAglOGp04RKcOlOBFKcMtOESnChYSGtOTCQipCBYANie2IxszDC0BYpMFIVnF7cZjhAAUkGSM285EpATZIJkjrJBMsBF0i705YukA6QK56cWenLOokWqJArKtOK1ElnVSLVKcCrqJFqiSyqJFaiSRXVEi7rH6iRaokIWtJI5SSCpLHaKQC0kjlJIOikdpJIEqaRqmkxSSNU1hLypDIkyiQ6LAwiQmzJqsmFgQVZLZkwJiowUEnIAXucgBzvygQaw3yi01rHhsNk9QbfBRdm7uyJqusuu5cmngt241iL//ADUj4m/TjNLL5liSzHMsTmT1O+RteY/luuK1zpZhVdc95Cm47xe585Clr2GXZsFI43v5eGU55iau0eEP/oyKQqZ7yD8LQnUb3T1xzz3c8j6WI+M27ROmExC3Q9Qd4++fH040GGzGNBaafC1VYZruZe477QXF20wbLBaOxS1UV0NwwDAjlyjLCSzLMsC6xphBOsBN1gKiR11gKiwK+okVqJLGokVqJArqqRSqks6iRSqkCsqJFqiyxqpFKiyRbUUj1FYCiseorIQNRSO00gaSxyksAlNY1TWDpiMIISmiwqCRUQgECYElaYElA9acu/ELWg1nbC4dv9tDaqwP7RhvW/6Ad/M9M9s1+08cJhrUzarVJRDxUfvuOgNgebCcdonOwkVfGfJlKPDz+kBiRwUfSGq1dns+Ld5PCHweHepkiljwAHxkW67rSY29RSPTIOc2lsLfABjb9plbpxlno38Pq9XtVeyOW8zbsbqnfDigCbDPLieBPxmWXNi1nBk5CuSnO/gDEmm2aa1Yq0AQRlwYehmosCCQd4mmOUynTLPC43tvn4Zac2XOFqHJrtSJ4NvZfEC/gZ0oz58w1dqbq6GzKwZT3g3neNE44YihTrAW9ogYjkbZjzlmWU+RyIJoZoJpKoLiAcRlhAsICtRYtUWPOIs6wEKixWossKixSqsCuqrFKqyxqrE6qyRb0Vj1FYtREeorIQYpLG6YgKSxqmISPTEOogkEMogTEmJESQgTBkpASQEDj/4maR9pjWW/ZoqtMe8Rtuf6gP5Zq+CbexhNYcUales977dWow6Fzs/C0XptYAc5DU9o3BPiq60kF2ZvAcye4Cd11a1ZpYZAAAW4sd5POaH+EOiwTUxLDcfZpl4sR8B4TrFJpycuW8tfh18c8cfsT2SgQFSmDCOYF5nV4qNK4NXUqwvOJ656HNCqSNx3Hn/7O64sTRtedG+1pHmMx1jjy8ck54+eOnIFadS/CzSG1QqUDvpttL7tTP8AuDec5Y4sSDwm3fhni9jGBb5VKboeo7Q/tPnPQeffTrLGDMmZEwzDMgwhTIGAuwgaixl4B4CdQRWqsdqRWoICFVYnVWWFQROqIFvREdoiJ0Y7ShBulGqcVpRpISYWFWBUwitAKDMiQvPAwDAwWMq7NN25Ix8gTPbUr9Ya2zha55Un/tMEcEr7x3CSBzty+QmK47XkJ4ZnzkNr7d31Cwy0sHRW4uV2mzH5n7R9Zs5qgcc5yfDY7FilSTD4FXAQBnsVNxy2e1nzHOTXWbG0idqjUCjerj2lvddbN5gzi8Mruu3ym9OpLVvBvWtK7QWkEq01dTYst7NcZ8bNxEpdatYBh87F24KL2HUgTLu3TTUbFWq7Zsu+a7rDbZIuJqdTWPGt2RQYBiDxRLG2bNYlt/MSOjnrV22auHFNlJu9jYgcbnOaXjsm6rjnLdRomlqWzWPWO6qVtjF0DyqAf8hb5yWtdHYr2HKJ6LbZr0yOFSmR/wAhOzG7xjizmsrHdrzF4Om9wJnal3OyZBjPFpBjAixgXhGg2gL1BFqgjTxepATqCKVRHakUqwLOlHKUTpRylCDVOMoYrTMYQwkwphAYBTCAwChpnagwZ68Al5Ta3VLYSqOa28yAfhLa81/XRr4dxyAPxt85F9Jx9uP1VzkcPmw6/wDYRnELa/397orTyYffESPhvPb6K0RhAaSWOz2V3W4DLxjVTRpGbVC3IW+ZJ+Ep9X9J7VJD3D0lhpTSLBbU83bJe6/GedvT0LjdnEp7Nyu47wOfE9ZTYmgPabdQ3b9xf03yJtuvbK9pbYJgBYG+QzPE8SZr+m3XbuW2XBQUzxzY5d8jRFpXwrMBaoF601J8DcekrMVh1S5uSeJbefoJYLjDsZjO3xmuaXx5sY9rTFzPXartYgeMrMKbVFPJlPkR9I5rGt6oPX5WiuHXtDqPWejj/GPP5O867VQbsjoPpJ7UUwjdhfdENeXcohMiTMXnrwMGQaSJkGhITxeoId4B4CtSK1BG6kVqQhYUo3TMTpRunAaSHQxZIdTAMIQGBUyYMAt568heevAJtTX9cT/sOO5f7r/KXl5Ra2fsXPcPn9ZGXpbD25fiFv4n79Yq6bz3fSPst/v75Qb08m8fkJXbbTrGpbXoL0mxYfDja2jvG7/yaZ+H+LBpBSd2UvNZP9Q1K2GYLdgG4MUv2gp4G08/Kf8AWno49o6YwbU2L4Woqsb7SlwoPgTvmv6PRPbe1xVemXB7KhrgeO6bJo/VbAGltVTU2jTX87G4cgm+RtfLcMsjFNZtW8MFK4SlsvsqNtiwVbMCxN95IP8AiaeGp7JyY2+OrtbuoK3E1bSVK5PdL7DP7OmAzbVhba3XtNS0/pVaaMxP39ZnhLb0m3x9tH04b1mtwsPIE/ODw9PtDqPiZFLt223tdvO9vhGsIvbX3l9bz0PU0863d26bgn7C9BGA0Qwb9kfe+MhpZzmLzN4INJAyRImQYzxMiTAg8A8M8A8BepFakZqRWpCD1IxumYlSMbpmA2hh0iyGHQwDCTEGpkwYE7z0iDMwM3lJrQL0anS/l/mXN5UaxfsKnuNIy9LY+3NE3t3W+cwVymEOTHmRMsd3X5GZunSx1W0r7CqVY9ktv5GdRr3dDsnJhcH4iciweF2ye8zetWNLNQtSr5puVz+7/C3d3zm5ce9x1cVuj9PTapda6sG42Ngbbj1tFcXpg4hlp0Qdm/abnNrqGi4syIeoHmItUSjRG3soijMnISm3T/t/Xai0vivY0mJNrZDvM5dprEtXYD92+Q595mz6fx7Yup2cqYJ2Rz7zKWvhtkg8vXh99024cfH7cfNbZoArbyt6D6wuC/aL1Eg3HwHlM4Z7VRfnabufTfMI/ZHQRtWlVg6uQ6f4MepvLxz04pkwYsrQoaSgS8wTI3kSYHmMA5hGMC5gBqGK1DGKhitQwg5SMbpmIUjG6bQHaZh1MUpmMIYDKmSBgVMmpgF2pB64GZBI5gE/ARw6oV8RYlzQHPLaI935G0p9Kao43CN7Wk9DEqL3TEmpsHvKBgpPUyly0vjjL8h4rWDDpce0DNu2VuzX5bIBM1zWDTDMlqivSVgwRCtnfvJbILz2drqDlMaX/E7HU70qaYTD2yvQphivuksVHkZpbYupXc1KzvUdjmzksx5C58cpF3WmOMl9GQMvP1mWz++v1ma9MrcMLEc9++8FTbI+HyldttLvQC55/Zm8jRwqJYjIiarq3QDZHj8gLToWi6eVm32F/qJy53eTpnUaVicLVpEqrsF5XNvKK18M9QWYk9TN2xmAu5FpFdEEbxK+VX3Go0NHbC3IlHpT83Sb9pvDbC+Fz3AfYHjOf6TN7nn9n4+k047uqZzpWr8/nf6xfb7V+8nyhar2+/CAoi7AHp5/ZnS5a69q/q5h8Zh0rYd6lMkZj9ogbiCDnv5GRxWrmKpH9n7QDjT7X9H5vIGatqTrM+jKuzUBNNrbS7v5l752/Ruk6OJRXpOCGzGZuO6Rjl8MuTGzv4cpD2JU3BG8EEEdQcxCq06tjdG0a4tWpK/IkdodG3jwM13H6kUznh6jIf0v218DvHxmkrFpwMwTLDSuhK2GzYBk/Wu7l2hvHpnvlZtSR5jBOZJmgXaSBVDFahh6hitQwgzSaN02lfSaN02kh+m0YRolTaMI0gP4Wk1RlRBdmNh9T3TddDaNp0wCFBPByM+VweF99hz8ZSarYTsNUO9rqPcH5rdTl/LNrobrSKGFkMRQWopV1DA5FSNoeIkWqHcoue/JR9T95SQVjkzeC9n43v8AGVTGhae1BLsWwlPB0eJqVKJqsO9VclQe+IaL1Ao4LaxOJqDEVBmu5UL52so3W39wBnTtgDhu55+s0LXzSeySiMA1rPUOaUkyuo/U5yJ4mwHTPPqNuO3K6cp08t6j+8bkc+PrKimPv76TadK6MZaa1GVlVvyB/wA9v1PyLb7TXEpXcjp8yJTG9OqztuuqNIM38oYek3+klus5zqdiwCt8ip2SOasLX8CB5zpeGW9sphl7a31BnobWfGSUj97hJ7FovjDYG+4Aseg+/hIRGraz4gEm+4WJH9oy5X82E0HHpe5+7ffpNi0xULEC/wCZrnpvP/UeErMZh7qeh+Zk4VtcOmn4o+o+szhVDOFJtfIHkeEZxWGIco2RvbPgeF+6BfDkHO4z8jOuXpw2dtt0TQp1XSni6e0VYBkBIdkIttUz+8L2OXLkZ1XQOqmDo2qYZqgvYkCowHiL3HQzRtRMRTxAWhjaQbZP+3Ut8xmDOtYXCIgGzfxJPrKYzdV5ctTQqbKjj43PxMltA7jfpPFoMKCdq2fObOUvi0ByIByNwRcEHIgjiJoOsuhv9OQ9P9k5sBv2G37N+I32Pd579XbeT0EqtMUPa0KlPjskr7y5r8QJaIc5ZoJ2mNuCdpIhUaLVGhKjRao0kHpPG6bytpNG6TwLGm0OHiFN4zSNyBzIHnIHS9HpsU0Xkqj4Z/G8sKTyvR41SeQHbwqmKq14VGkAzC4ib6JomxampKm4uL58+sbBnmbKRZKmWz05V+KNcK6J1YDnbeen1mn0NHZq1r+0qkDoikn+6dM09qmMVWNSoxN7Bj/AD+ReS8+JJ7pV6O0GDWqU2FtgE0xwG1lceCrOa7278MsfH6adhML7MCoMh7RkPcfzIT1z/wCM6noGtt0wfv7BuPCathNEZ16Di5YBk72TtC3UMwj+qFcoTRc5g3W/Ec+v175lle9tveN18Nw9nKzTKWpVD/DbwNvqZa7UVx9PaR15rIy9KY3tzt8PtVrW3Kflf0lrS0OGZVtcHf0UXI8RtfCZqUrVVP6lPna82/Q+EDAsegPI779ch5ynHPLKRty8nhjtyzXXV0gCtSBIAAewvla6t35XB6CJ6A0UmLAVWC1RkUe+y45q3Azr2IwgD9sWvl/Cwzyz3HM5d53ytfVKklT22H7N/wAyfut9DOrxrknLPktqfqu2HY7YFu/f9DN2AsLCLYK4AGfQ5/GMOZthjJHLyZ3K9oVWsJGobDLpB1Wuyjx8v8zFZ8+mcuzAxDZheQufQfM+EVZ98kGuC36jfw4fCKVH3wlzLatBO09WbtN1PrAM8shh2i1RpN2i1RpILTaNU3ldTeM03gWNN41SqWIPIg+RldTeHV8oHUqdSNUqkpsJWuo6D0j1KpKi1R4dWlbTqxqnUkB5Wk4rTeGR5I84++kqEwdq+0N5Qnr2rCXDSAXO/hKZY7Wxy01zSNDYalWG9GAJ/hIsbxTSej9hy6ZEEMvQ7x03j+WX+lMNtUyPH43i9dduilQ7wAr+h/qHxM5+Xj3K6uLl1Z/SeHq3FzDysR9kRwVN3SYStrFbpLR2RK7w20vzHjnNh0ZR2Ka94uepilFdpgvDj04yzLTfgw7uTD/I5LZMWagByIuIuKQX8u6FZoNp06cu00aZdoLagq9awJlkI0nu7HlZR6n1gcTUvl+o5+6N/wAMvGQosQl+JufEwAfefAdB9T8pAnWqWErq1SwMLiKnDxMrsVVyMDndd+03vN6mAZ5iu/ab3m9TAM8uJO8Xdpl3i7tAJTeMU3iCNGEeBYU3h1eV6PDq8Douj63YXoPSWNOrNd0dW7C+6PSWlKrIFwlSNUK0p6dWM06sgXKVIdKkq0rQ61YFkKkmHiC1YVasBl1vFUpgBkP5Tf4wgqzDteRYba7XcrdW3qSD4cYajWuF6T2nqBt7ReAs3Tg3h97pVaMxBcoo3m4/qM8/PG45aelhlMsNtr0aMi3gPn990aDQC2UBRuGUyHndhj4zTz88vK7MXgneDarAVKsuqIakSxta4tzNvrI1a0SerdoDVarkAOOX/sG9S3hFjVu3Qev+IDE1uHP0kCTVb3PP0ldjKuRha1WVeLrSRo2Jftt7zepgS8HWqXZvePqYIvLAjPAs0wzwTNJE0MOhnp6AwhhlM9PSBt+jz2F91fQSzpGenpAcpGMUzPT0ihumYwhnp6SDKYZTMT0AgMzeenpAwZr/ALJaGNppSAVSl7DPfe++ZnpnyT19r4W9z9L0GZJnp6aKBuYvUaYnpITrMYqzZzE9IGEO/r8hF3OZnp6ArWMq8acjPT0kc/Y5yJnp6WEDBsZ6egf/2Q==')